# ATM Project

In [10]:
import random,getpass,os,sqlite3,time,sys


## 一.創建DB

In [11]:
class Database():
    # 1.1 創建資料表
    def create_database(self):
      conn = sqlite3.connect(database='ATM_data.db')
      #conn.execute("DROP TABLE USERS; ") 
      # ID:帳號、Name:姓名、idCard:身份證號、Phone：手機號碼
      conn.execute("""CREATE TABLE IF NOT EXISTS USERS
            (ID TEXT PRIMARY KEY NOT NULL,
            Name TEXT(12)     NOT NULL,
            idCard TEXT    NOT NULL,
            Password INT(12)   NOT NULL,
            Phone INT(10)       NOT NULL,
            Momey REAL,
            Lock INT);""")
      conn.commit()
      # ID:帳號、Name:姓名、card_id:卡片號碼、card_lock：鎖卡機制
      conn.execute("""CREATE TABLE IF NOT EXISTS Card
            (ID TEXT PRIMARY KEY NOT NULL,
            Name TEXT(12)  NOT NULL,
            card_id TEXT    NOT NULL,
            card_passwd INT(12)  NOT NULL,
            card_lock INT);""")
      conn.commit()
      conn.close()

    # 1.2 加入資料
    ## 1.2.1 User_Data
    def Insert_User_db(self,user_ID, user_Name, user_idCard, user_Password, user_Phone, user_Momey=0):
      conn = sqlite3.connect('ATM_data.db')
      lock = 0
      cmd = "INSERT INTO USERS (ID,Name,idCard,Password,Phone,Momey,Lock) VALUES (\'{}\',\'{}\',\'{}\',{},{},{},{})".format(user_ID, user_Name, user_idCard, user_Password, user_Phone, user_Momey,lock)
      conn.execute(cmd)
      conn.commit()
      conn.close()

    ## 1.2.2 User_Card_Data
    def Insert_Card_db(self,user_id, card_id):
      conn = sqlite3.connect('ATM_data.db')
      user_Name = conn.execute("SELECT Name FROM USERS WHERE ID = \'{}\'".format(user_id))
      card_passwd = conn.execute("SELECT Password FROM USERS WHERE ID = \'{}\'".format(user_id))
      print("user_id：",user_id,"\ncard_Name：",user_Name,"\ncard_passwd：",card_passwd)
      card_lock = 0
      cmd = "INSERT INTO Card (ID,Name,card_id,card_passwd,card_lock) VALUES (\'{}\',\'{}\',\'{}\',{},{})".format(user_id, user_Name, card_id, card_passwd, card_lock)
      conn.execute(cmd)
      conn.commit()
      conn.close()

    # 1.3 刪除資料
    def Del_db(self,user_ID):
      conn = sqlite3.connect('ATM_data.db')
      cmd = "DELETE FROM USERS WHERE ID = \'{}\'".format(user_ID)
      conn.execute(cmd)
      conn.commit()
      cmd = "DELETE FROM Card WHERE ID = \'{}\'".format(user_ID)
      conn.execute(cmd)
      conn.commit()
      conn.close()
      print("刪除成功")

    # 1.4 查詢餘額
    def select_db(self,user_ID):
        conn = sqlite3.connect('ATM_data.db')
        money = conn.execute("SELECT Momey FROM USERS WHERE ID = \'{}\'".format(user_ID))
        for i in money:	
          print('餘額:{}元'.format(i))	
          conn.commit()	
          conn.close()	
          return i	

    # 1.4 查詢他人餘額
    def select_other_db(self,other_ID):
        conn = sqlite3.connect('ATM_data.db')
        money = conn.execute("SELECT Momey FROM USERS WHERE ID = \'{}\'".format(other_ID))
        for i in money:	
          print('餘額:{}元'.format(i))	
          conn.commit()	
          conn.close()	
          return i
        

    # 1.5 變更密碼
    def updata_db(self,user_ID, user_password):
        conn = sqlite3.connect('ATM_data.db')
        conn.execute("UPDATE USERS SET Password = {} WHERE ID = \'{}\'".format(user_password, user_ID))
        print('變更密碼完成...')
        conn.commit()
        conn.close()

    def updata_Card_db(self,user_ID, user_password):
        conn = sqlite3.connect('ATM_data.db')
        conn.execute("UPDATE Card SET card_passwd = {} WHERE card_id = \'{}\'".format(user_password, user_ID))
        print('變更密碼完成...')
        conn.commit()
        conn.close()

    # 1.6 提款
    def withdraw(self,user_ID, with_money): 
        db=Database()
        conn = sqlite3.connect('ATM_data.db')
        user_money = db.select_db(user_ID) 
        if user_money[0] - with_money > 0:
          conn.execute("UPDATE USERS SET Momey = \'{}\' WHERE ID = \'{}\'".format(user_money[0] - with_money, user_ID))
          print('本次取出：{}元\n帳戶餘額：{}元.'.format(with_money, user_money[0] - with_money))
          conn.commit()
          conn.close()
        else:
          print("帳戶餘額不足，操作失敗...")
          conn.commit()
          conn.close()

    # 1.7 存錢
    def wallet(self,user_ID, wallet_money): 
        db=Database()
        conn = sqlite3.connect('ATM_data.db')
        user_money = db.select_db(user_ID)
        conn.execute("UPDATE USERS SET Momey = \'{}\' WHERE ID = \'{}\'".format(user_money[0] + wallet_money, user_ID))
        print('本次存入：{}元\n帳戶餘額：{}元.'.format(wallet_money, user_money[0] + wallet_money))  
        conn.commit()
        conn.close()

    # 1.8 轉帳
    def transfer_accounts(self,user_ID, other_ID, turn_money):
      verification=Verification()
      db=Database()
      if other_ID == user_ID:
        print('錯誤，不能給自己轉帳')
        return None
      conn = sqlite3.connect('ATM_data.db')
      user_money = db.select_db(user_ID)
      other_money = db.select_other_db(other_ID)  
      if user_money[0] - turn_money > 0:
        conn.execute("UPDATE USERS SET Momey = \'{}\' WHERE ID = \'{}\'".format(user_money[0] - turn_money, user_ID))
        conn.execute("UPDATE USERS SET Momey = \'{}\' WHERE ID = \'{}\'".format(other_money[0] + turn_money, other_ID))
        print('轉出:{}元\n帳戶餘額：{}元'.format(turn_money, user_money[0] - turn_money)) 
        conn.commit()
        conn.close()
      else:
        print("帳戶餘額不足，操作失敗...")
        conn.commit()
        conn.close()

    # 1.9 鎖定帳戶
    def lock(self,user_ID):
      conn = sqlite3.connect('ATM_data.db')
      lock = 1
      conn.execute("UPDATE USERS SET Lock = {} WHERE ID = \'{}\'".format(lock,user_ID))
      print('帳戶已鎖定')
      conn.commit()
      conn.close()

    # 1.10 解鎖帳戶
    def open_look(self,user_ID):
      conn = sqlite3.connect('ATM_data.db')
      lock = 0
      conn.execute("UPDATE USERS SET Lock = {} WHERE ID = \'{}\'".format(lock,user_ID))
      print('帳戶已解鎖')
      conn.commit()
      conn.close()

    # 1.11 補卡
    def re_card(self,user_ID, Card_ID):
      conn = sqlite3.connect('ATM_data.db')
      conn.execute("UPDATE Card SET card_id = \'{}\' WHERE ID = \'{}\'".format(Card_ID,user_ID))
      print('重新辦卡成功...')
      conn.commit()
      conn.close()

## 二.註冊帳戶

### 2.1 驗證

In [12]:
class Verification():
  #確認資料庫是否有此帳號，已存在True,否則False
  def checking_User_ID(self, user_ID):
    conn = sqlite3.connect('ATM_data.db')	
    check = conn.execute("SELECT ID FROM USERS WHERE ID = \'{}\'".format(user_ID))
    for i in check:	
      if i[0] == user_ID:	
        conn.commit()
        conn.close()
        return True
      else:
        conn.commit()
        conn.close()
        return False
    else:
        return False

  #確認使用者登入是否正確
  def checking_User_password(self, user_ID, user_password):
    conn = sqlite3.connect('ATM_data.db')
    check = conn.execute("SELECT Password FROM USERS WHERE ID = {}".format(user_ID))
    for i in check:
      if i[0] == str(user_password):
        conn.commit()
        conn.close()
        return True
      else:
        conn.commit()
        conn.close()
        return False
  

  #確認資料庫是否有此卡號，已存在True,否則False
  def checking_Card_ID(self, user_ID):
    conn = sqlite3.connect('ATM_data.db')	
    check = conn.execute("SELECT card_id FROM Card WHERE ID  = \'{}\'".format(user_ID))
    for i in check:	
      if i[0] == user_ID:	
        conn.commit()
        conn.close()
        return True
      else:
        conn.commit()
        conn.close()
        return False
    else:
        return False


  def checking_Card_password(self, user_ID, user_password):
    conn = sqlite3.connect('ATM_data.db')
    check = conn.execute("SELECT card_passwd FROM Card WHERE card_id = \'{}\'".format(user_ID))
    for i in check:
      if i[0] == str(user_password):
        conn.commit()
        conn.close()
        return True
      else:
        conn.commit()
        conn.close()
        return False


In [13]:
class Verification():
  #確認資料庫是否有此帳號，已存在True,否則False
  def checking_User_ID(self, user_ID):
    conn = sqlite3.connect('ATM_data.db')	
    check = conn.execute("SELECT ID FROM USERS WHERE ID = \'{}\'".format(user_ID))
    for i in check:	
      if i[0] == user_ID:	
        conn.commit()
        conn.close()
        return True
      else:
        conn.commit()
        conn.close()
        return False
    else:
        return False

  #確認使用者登入是否正確
  def checking_User_password(self, user_ID, user_password):
    conn = sqlite3.connect('ATM_data.db')
    check = conn.execute("SELECT Password FROM USERS WHERE ID = \'{}\'".format(user_ID))
    for i in check:
      if i[0] == str(user_password):
        conn.commit()
        conn.close()
        return True
      else:
        conn.commit()
        conn.close()
        return False
  

  #確認資料庫是否有此卡號，已存在True,否則False
  def checking_Card_ID(self, user_ID):
    conn = sqlite3.connect('ATM_data.db')	
    check = conn.execute("SELECT card_id FROM Card WHERE ID  = \'{}\'".format(user_ID))
    for i in check:	
      if i[0] == user_ID:	
        conn.commit()
        conn.close()
        return True
      else:
        conn.commit()
        conn.close()
        return False
    else:
        return False


  def checking_Card_password(self, user_ID, user_password):
    conn = sqlite3.connect('ATM_data.db')
    check = conn.execute("SELECT card_passwd FROM Card WHERE card_id = \'{}\'".format(user_ID))
    for i in check:
      if i[0] == str(user_password):
        conn.commit()
        conn.close()
        return True
      else:
        conn.commit()
        conn.close()
        return False
  
  def checking_Lock(self, user_ID):
    conn = sqlite3.connect('ATM_data.db')
    check = conn.execute("SELECT Lock FROM USERS WHERE ID = \'{}\'".format(user_ID))
    if check == 1:
        return True
    else:
      return False


### 2.2 隨機生成ID

In [14]:
class Creat_ID():
  # 隨機生成帳號
  def Creat_User_ID(self):
    verification = Verification()
    while True:
      str = ""
      for i in range(10):
        ch = chr(random.randrange(ord('0'), ord('9')+1))
        str += ch
      # 判斷是否重複
      if not verification.checking_User_ID(str):
        return str
      else:
        continue       

  # 隨機生成卡號
  def Creat_Card_ID(self):  
    verification = Verification()
    while True:
      str = ""
      for i in range(16):
        ch = chr(random.randrange(ord('0'), ord('9')+1))
        str += ch
        # 判斷是否重複
      if not verification.checking_Card_ID(str):
        return str
      else:
        continue  

### 2.3 開戶

In [15]:
def register_user():
  creat_ID = Creat_ID();
  db = Database()
  while True:
    try:
      print("請開始開戶流程：\n")
      user_name = str(input("請輸入您的姓名："))
      idCard = str(input("請輸入您的身份證號："))
      phone = int(input("請輸入您的電話號碼："))
      # 設置密碼
      user_password = getpass.getpass("請輸入您的密碼：")
      user_password_2 = getpass.getpass("請再次輸入您的密碼：")
      # 驗證密碼
      if user_password_2 == user_password:
        user_ID = creat_ID.Creat_User_ID()
        Money = int(input("請輸入您的預存款金額："))
        if Money < 100:
          print("預存款輸入少於100元，開戶失敗，請重新嘗試......")
          continue
        else:
          db.Insert_User_db(user_ID, user_name, idCard, user_password, phone, Money)
          print("開戶成功，請牢記帳號:'%s'...."%user_ID)
          return True
      else:
        print("密碼輸入有誤，開戶失敗，請重新嘗試......1")
        continue
  
    except:
        print('輸入錯誤，開戶失敗，請重新嘗試......2')
        return False
    listA = [user_ID, user_name, idCard, user_password, Money]
    print(listA)
    return True

## 三. Admin

In [16]:
class Admin():
    def __init__(self):
      self.admin = "Admin"
      self.passwd = "Admin"

    # 管理員驗證
    def login(self):
        inputAdmin = str(input("請輸入管理員帳戶: "))
        if inputAdmin != self.admin:
          print("帳號輸入錯誤！")
          return False
        inputPasswd = getpass.getpass("請輸入管理員密碼: ")
        if inputPasswd  != self.passwd:
          print("密碼輸入錯誤！")
          return False
        print("操作成功,請稍後...")
        time.sleep(2)
        return True
  
    # Admin 歡迎
    def View(self):
      print("**********************************************")
      print("*                                            *")
      print("*                                            *")
      print("*              歡迎登入 QAQ 銀行            *")
      print("*                                            *")
      print("*                                            *")
      print("**********************************************\n")

    # Admin 功能界面
    def Function_View(self):
      print("\n********************************************")
      print("*           開戶（1）     查詢（2）         *")
      print("*           提款（3）     存款（4）         *")
      print("*           轉賬（5）     改密碼（6)        *")
      print("*           鎖定（7）     解鎖（8）         *")
      print("*           補卡（9）     銷戶（10）        *")
      print("*                   退出（0）               *")
      print("********************************************")


## 四. User

In [17]:
class User():
  # 登入
  def login(self):
    verification = Verification()
    db = Database()
    try:
      error_password = 0
      ckeck = str(input("是否有帳號：")) 		
      while ckeck == 'Yes' or '是':	
        self.user_id = str(input("請輸入您的帳號: "))	
        if verification.checking_Lock(self.user_id):
          print("該帳戶已被鎖定，請找專員解鎖")
          return False
        else:  
          self.user_password = getpass.getpass("請輸入您的密碼: ")
          if verification.checking_User_ID(user_id) and verification.checking_User_password(user_id,user_password):
            error_password = error_password + 1
            if error_password < 3:
              print('輸入帳號或密碼錯誤，登入失敗，請重新嘗試.......\n當輸入錯誤超過三次，即將鎖定帳戶，請小心服用...')
            else:
              db.look(self.user_id)
            continue
          else:
            return True
      while ckeck == 'No' or '否':
        register_user()
        return True
    except:	
      print('輸入錯誤，登入失敗，請重新嘗試.......2')
      return False

  # User 
  def User_View(self):
    print("**********************************************")
    print("*                                            *")
    print("*                                            *")
    print("*             歡迎使用 QAQ 銀行              *")
    print("*                                            *")
    print("*                                            *")
    print("**********************************************\n")

  # User 功能界面
  def User_Function_View(self):
    print("\n**********************************************")
    print("*      歡迎使用   請選擇功能                 *")
    print("*      開戶（1）     存款（2)                *")
    print("*      提款（3）     轉帳（4）               *")
    print("*      開卡（5）     改密碼（6）             *")
    print("*      查詢（7）     退出（0)                *")
    print("**********************************************")
  
  # 開卡
  def Creat_Card(self):
    database = Database();
    user_id = str(input("請輸入帳號："))
    card_id = creat_ID.Creat_Card_ID()
    database.Insert_Card_db(user_id,card_id)
    print('開卡成功，請牢記卡號：',card_id)

  # 改密碼
  def modify_information(self):
    database = Database();
    verification = Verification()
    try:
      old_password = getpass.getpass('請輸入你的舊密碼:')
      if verification.checking_User_password(self.user_id):
        new_password = getpass.getpass('請輸入你的新密碼:')
        new_password2 = getpass.getpass('請輸入再次你的新密碼:')
      else:
        return '輸入密碼錯誤，請重新嘗試...'
    except ValueError:
      return '輸入錯誤，請重新嘗試...'
    if new_password2 == new_password:
      database.updata_db(self.user_id,new_password)
    else:
      return '輸入密碼錯誤，請重新嘗試...'

In [18]:
if __name__ == '__main__':
  database = Database();
  verification = Verification()
  creat_ID = Creat_ID();
  admin = Admin();
  user = User();
  database.create_database()
  while True: 
    try:
      check = str(input("登入人員："))
    except(IOError, ValueError):
        print('輸入錯誤,請重新輸入...')
        continue
    if check == "Admin":
      if admin.login():
        admin.View()
        while True:
          admin.Function_View()
          choose = int(input("請選擇操作："))
          if choose == 1:
            register_user()
            continue
          elif choose == 2:
            user_id = str(input("請輸入查詢帳號："))
            database.select_db(user_id)
            continue
          elif choose == 3:
            user_id = str(input("請輸入提款帳號："))
            if verification.checking_User_ID(user_id):
              with_money = int(input("請輸入提款金額："))
              database.withdraw(user_id,with_money)
            else:
              print('查無此帳戶，請重新嘗試...')
            continue
          elif choose == 4:
            user_id = str(input("請輸入存款帳號："))
            if verification.checking_User_ID(user_id):
              wallet_money = int(input("請輸入存款金額："))
              database.wallet(user_id,wallet_money)
            else:
              print('查無此帳戶，請重新嘗試...')            
            continue
          elif choose == 5:
            user_id = str(input("請輸入帳號："))
            if verification.checking_User_ID(user_id):
              other_id = str(input("請輸入轉帳帳號："))
              if verification.checking_User_ID(other_id):
                turn_money = int(input("請輸入轉帳金額："))
                database.transfer_accounts(user_id, other_id, turn_money)
              else:
                print('查無此帳戶，請重新嘗試...') 
            else:
              print('查無此帳戶，請重新嘗試...') 
            continue
          elif choose == 6:
            print("即將變更密碼...")
            user_id = str(input("請輸入帳號："))
            if verification.checking_User_ID(user_id):
              user_password = getpass.getpass("請輸入新密碼：")
              database.updata_db(user_id, user_password)
            else:
              print('查無此帳戶，請重新嘗試...')
            continue
          elif choose == 7:
            print("即將鎖定帳戶...")
            user_id = str(input("請輸入帳號："))
            if verification.checking_User_ID(user_id):
              database.lock(user_id)
            else:
              print('查無此帳戶，請重新嘗試...')
            continue
          elif choose == 8:
            print("即將解鎖帳戶...")
            user_id = str(input("請輸入帳號："))
            if verification.checking_User_ID(user_id):
              database.open_look(user_id)
            else:
              print('查無此帳戶，請重新嘗試...')
            continue
          elif choose == 9:
            print("補辦新卡中...")
            user_id = str(input("請輸入帳號："))
            if verification.checking_User_ID(user_id):
              Card_id = creat_ID.Creat_Card_ID()
              database.re_card(user_id, Card_id)
              print("補辦完成，請牢記新卡號：",Card_id)
            else:
              print('查無此帳戶，請重新嘗試...')
            continue
          elif choose == 10:
            print("即將消除帳戶...")
            user_id = str(input("請輸入帳號："))
            if verification.checking_User_ID(user_id):
              database.Del_db(user_id)
            else:
              print('查無此帳戶，請重新嘗試...')
            continue
          elif choose == 0:
            print("正在退出，請稍定...")
            print("感謝使用，歡迎再次使用")
            break
          else:
            print("輸入錯誤，請重新輸入...")
            continue
      else:
         continue 
    elif check == "User":
      user.login()
      user.View()
      while True:
        user.Function_View()
        select = int(input("請選擇操作："))
        if select == 1:
          register_user()
          continue        
        elif choose == 3:
          user_id = str(input("請輸入提款帳號："))
          with_money = int(input("請輸入提款金額："))
          database.withdraw(user_id,with_money)
          continue
        elif choose == 2:
          user_id = str(input("請輸入存款帳號："))
          wallet_money = int(input("請輸入存款金額："))
          database.withdraw(user_id,wallet_money)
          continue
        elif choose == 4:
          user_id = str(input("請輸入帳號："))
          other_id = str(input("請輸入轉帳帳號："))
          turn_money = int(input("請輸入轉帳金額："))
          database.transfer_accounts(user_id, other_id, turn_money)
          continue
        elif choose == 5:
          user.Creat_Card()
          continue
        elif choose == 6:
          print("即將變更密碼...")
          user.modify_information()
          continue
        elif choose == 7:
          user_id = str(input("請輸入查詢帳號："))
          database.select_db(user_id,with_money)
          continue
        elif select == 0:
          print("正在退出，請稍定...")
          print("感謝使用，歡迎再次使用")
          break
        else:
          1
          print("輸入錯誤，請重新輸入...")
          continue
    else:
      print("輸入錯誤，請重新輸入...")
      continue

登入人員：Admin
請輸入管理員帳戶: Admin
請輸入管理員密碼: ··········
操作成功,請稍後...


KeyboardInterrupt: ignored

In [ ]:
db = Database();
db.create_database()
conn = sqlite3.connect('ATM_data.db')
data = conn.execute("SELECT * FROM USERS ")
for i in data:
  print(i)
conn.commit()
conn.close()